In [7]:
import open3d as o3d
import warnings
import pdb
import os
import numpy as np
import open3d as o3d
import re
import matplotlib.pyplot as plt
import scipy.stats as stats
import pandas as pd
from voxel_grid import *


# Visualize the point cloud
# o3d.visualization.draw_geometries([point_cloud])
import numpy as np
import os
from collections import defaultdict
import matplotlib.pyplot as plt
import open3d as o3d
import numpy as np
from point_cloud_FoV_utils import *
import pandas as pd
from tqdm import tqdm

In [34]:
# participant = 'P01_V1'
# trajectory_index = 0
image_width, image_height = np.array([1920, 1080])
# generate graph voxel grid features
voxel_size = int(64)
min_bounds = np.array([-251,    0, -241]) 
max_bounds = np.array([ 262, 1023,  511])

edge_prefix = str(voxel_size)
# get the graph max and min bounds
# graph_max_bound,graph_min_bound,graph_voxel_grid_integer_index_set,graph_voxel_grid_index_set,graph_voxel_grid_coords,original_index_to_integer_index = voxelizetion_para(
    # voxel_size=voxel_size, min_bounds=min_bounds, max_bounds=max_bounds)
results = voxelizetion_para(voxel_size=voxel_size, min_bounds=min_bounds, 
                            max_bounds=max_bounds)
graph_max_bound = results['graph_voxel_grid_max_bound']
graph_min_bound = results['graph_voxel_grid_min_bound']
graph_voxel_grid_integer_index_set = results['graph_voxel_grid_integer_index_set']
graph_voxel_grid_index_set = results['graph_voxel_grid_index_set']
graph_voxel_grid_coords = results['graph_voxel_grid_coords']
graph_voxel_grid_coords_array = results['graph_voxel_grid_coords_array']
original_index_to_integer_index = results['original_index_to_integer_index']
# for pcd_name in ['longdress','loot','redandblack','soldier']:
for pcd_name in ['soldier']:
    history = 60
    # future = 60
    # prefix = f'{pcd_name}_VS{voxel_size}_LR' # LR is _LR for testing***********************************************
    # prefix = f'{pcd_name}_VS{voxel_size}_TLR' # LR is _LR for testing***********************************************
    prefix = f'{pcd_name}_VS{voxel_size}'
    for future in [60]:
        # print(f'Processing {pcd_name} with history {history} and future {future}...')
        for user_i in tqdm(range(1,15)):  # TLP/LR is 15 for testing***********************************************
        # for user_i in tqdm(range(1,28)):                
            participant = 'P'+str(user_i).zfill(2)+'_V1'
            node_index = []
            occupancy_feature = []
            in_FoV_feature = []
            occlusion_feature = []
            distance_feature = []
            coordinate_feature = []
            # choose different trajectory files***********************************************
            positions,orientations = get_point_cloud_user_trajectory(pcd_name=pcd_name,participant=participant)
            # positions,orientations = get_point_cloud_user_trajectory_LR(pcd_name=pcd_name,participant=participant,history=history,future=future) # LR is _LR for testing***********************************************
            # positions,orientations = get_point_cloud_user_trajectory_TLR(pcd_name=pcd_name,participant=participant,history=history,future=future) # TLR is _TLR for testing***********************************************
            for trajectory_index in tqdm(range((len(positions)))):
                # print(f'Processing trajectory {trajectory_index}...')
                # Load the point cloud data
                pcd = get_pcd_data(point_cloud_name=pcd_name, trajectory_index=trajectory_index%150)
                # get the position and orientation for the given participant and trajectory index
                
                position = positions[trajectory_index]
                orientation = orientations[trajectory_index]
                para_eye = [i*1024/1.8 for i in position]
                para_eye[2] = -para_eye[2]
                # para_eye = np.array(para_eye).reshape(3,1)
                pitch_degree, yaw_degree, roll_degree = orientation
                
                # Define camera intrinsic parameters
                intrinsic_matrix = get_camera_intrinsic_matrix(image_width, image_height)
                # Define camera extrinsic parameters
                extrinsic_matrix = get_camera_extrinsic_matrix_from_yaw_pitch_roll(yaw_degree, pitch_degree, roll_degree, para_eye)



                # pcd = pcd.voxel_down_sample(voxel_size=8)
                # get the occupancy feature
                occupancy_dict,occupancy_array = get_occupancy_feature(pcd,graph_min_bound,graph_max_bound,graph_voxel_grid_integer_index_set,voxel_size)
                # print('occupancy_dict:      ',occupancy_dict[(1, 5, 2)])
                

                # get the in_FoV_voxel_percentage_dict
                in_FoV_percentage_dict,in_FoV_voxel_percentage_array,pcd_N = get_in_FoV_feature(graph_min_bound,graph_max_bound,voxel_size,intrinsic_matrix,extrinsic_matrix,image_width,image_height)
                # print('in_FoV_dict:         ',in_FoV_percentage_dict[(1, 5, 2)])

                # get occlusion level
                # deep copy the pcd
                
                occlusion_level_dict,occulusion_array,pcd_hpr = get_occlusion_level_dict(pcd,para_eye,graph_min_bound,graph_max_bound,graph_voxel_grid_integer_index_set,voxel_size,intrinsic_matrix,extrinsic_matrix,image_width,image_height)
                # print('occlusion_level_dict:',occlusion_level_dict[(2, 0, 2)])
                # print('occupancy_dict:      ',occupancy_dict)
                # print('occupancy_array:      ',occupancy_array)
                # print('occlusion_level_dict:',occlusion_level_dict)
                # print('occulusion_array:    ',occulusion_array)
                # print('in_FoV_dict:         ',in_FoV_percentage_dict)
                # print('in_FoV_array:        ',in_FoV_voxel_percentage_array)
                # visualize the voxel grid
                visualize_voxel_grid(pcd,pcd_hpr,graph_min_bound,graph_max_bound,voxel_size,para_eye,graph_voxel_grid_integer_index_set,graph_voxel_grid_coords)
                # append features
                w
                
                occupancy_feature.append(occupancy_array)
                in_FoV_feature.append(in_FoV_voxel_percentage_array)
                occlusion_feature.append(occulusion_array)
                node_index.append(graph_voxel_grid_integer_index_set)
                coordinate_feature.append(graph_voxel_grid_coords_array)
                distance_feature.append(np.linalg.norm(graph_voxel_grid_coords_array-para_eye,axis=1).reshape(-1,1))
                
            # save the features to the csv file
            occupancy_feature = np.array(occupancy_feature).reshape(-1,1)
            in_FoV_feature = np.array(in_FoV_feature).reshape(-1,1)
            occlusion_feature = np.array(occlusion_feature).reshape(-1,1)
            node_index = np.array(node_index).reshape(-1,1)
            coordinate_feature = np.array(coordinate_feature).reshape(-1,3)
            distance_feature = np.array(distance_feature).reshape(-1,1)
            # save to ./data/voxel_size256/node_feature.csv and column name is 'occupancy_feature','in_FoV_feature','occlusion_feature'
            node_feature = np.concatenate((occupancy_feature,in_FoV_feature,occlusion_feature,coordinate_feature,distance_feature,node_index),axis=1)
            node_feature_df = pd.DataFrame(node_feature,columns=['occupancy_feature','in_FoV_feature','occlusion_feature','coordinate_x','coordinate_y','coordinate_z','distance','node_index'])
            # if not os.path.exists(f'./data/{prefix}'):
            #     os.makedirs(f'./data/{prefix}')
            # # node_feature_df.to_csv(f'./data/{prefix}/{participant}node_feature.csv')
            #     # LR for testing***********************************************
            # node_feature_df.to_csv(f'./data/{prefix}/{participant}node_feature{history}{future}.csv')

min_bounds: [-251    0 -241]
max_bounds: [ 262 1023  511]
graph max_bound: [ 324 1023  526]
graph min_bound: [-251    0 -241]
original_index_to_integer_index: {(0, 0, 0): 0, (0, 0, 1): 1, (0, 0, 2): 2, (0, 0, 3): 3, (0, 0, 4): 4, (0, 0, 5): 5, (0, 0, 6): 6, (0, 0, 7): 7, (0, 0, 8): 8, (0, 0, 9): 9, (0, 0, 10): 10, (0, 0, 11): 11, (0, 1, 0): 12, (0, 1, 1): 13, (0, 1, 2): 14, (0, 1, 3): 15, (0, 1, 4): 16, (0, 1, 5): 17, (0, 1, 6): 18, (0, 1, 7): 19, (0, 1, 8): 20, (0, 1, 9): 21, (0, 1, 10): 22, (0, 1, 11): 23, (0, 2, 0): 24, (0, 2, 1): 25, (0, 2, 2): 26, (0, 2, 3): 27, (0, 2, 4): 28, (0, 2, 5): 29, (0, 2, 6): 30, (0, 2, 7): 31, (0, 2, 8): 32, (0, 2, 9): 33, (0, 2, 10): 34, (0, 2, 11): 35, (0, 3, 0): 36, (0, 3, 1): 37, (0, 3, 2): 38, (0, 3, 3): 39, (0, 3, 4): 40, (0, 3, 5): 41, (0, 3, 6): 42, (0, 3, 7): 43, (0, 3, 8): 44, (0, 3, 9): 45, (0, 3, 10): 46, (0, 3, 11): 47, (0, 4, 0): 48, (0, 4, 1): 49, (0, 4, 2): 50, (0, 4, 3): 51, (0, 4, 4): 52, (0, 4, 5): 53, (0, 4, 6): 54, (0, 4, 7): 55, (0

  0%|          | 0/14 [00:06<?, ?it/s]

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


NameError: name 'w' is not defined

In [33]:
%run voxel_grid.py
visualize_voxel_grid(pcd,pcd_hpr,graph_min_bound,graph_max_bound,voxel_size,para_eye,graph_voxel_grid_integer_index_set,graph_voxel_grid_coords)

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [37]:
voxel_size = int(256/4)  # You can adjust this size as needed
min_bounds = np.array([-251,    0, -242])
max_bounds = np.array([ 262, 1023,  512])
# (512+242)/128 = 5.891
# (262+251)/128 = 4.007
# (1023+0)/128 = 7.992 
voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud_within_bounds(pcd_hpr, voxel_size, min_bounds, max_bounds)

In [38]:
o3d.visualization.draw_geometries([voxel_grid])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
